## VARIOUS DEFINITIONS OF FAIRNESS COMPARED

Here we will compare the aspects of the different definitions of Fairness on the [Adult Dataset](https://archive.ics.uci.edu/ml/datasets/adult).  

### IMPORTING THE NECESSARY LIBRARIES

We will be using the open-source [AIF360](https://github.com/Trusted-AI/AIF360) package.

In [3]:
import numpy as np
np.set_printoptions(suppress = True)

import matplotlib.pyplot as plt

# Importing the Dataset
from aif360.datasets import AdultDataset
from aif360.algorithms.preprocessing.optim_preproc_helpers.data_preproc_functions import load_preproc_data_adult

from aif360.datasets import BinaryLabelDataset
from aif360.metrics import BinaryLabelDatasetMetric, ClassificationMetric

from aif360.metrics.utils import compute_boolean_conditioning_vector
from common_utils import compute_metrics

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

### DATASET

The following shows the raw data.

In [5]:
data_adult = AdultDataset()
data_adult

               instance weights features                ...                                                  labels
                                                        ...                                                        
                                     age education-num  ... native-country=Vietnam native-country=Yugoslavia       
instance names                                          ...                                                        
0                           1.0     25.0           7.0  ...                    0.0                       0.0    0.0
1                           1.0     38.0           9.0  ...                    0.0                       0.0    0.0
2                           1.0     28.0          12.0  ...                    0.0                       0.0    1.0
3                           1.0     44.0          10.0  ...                    0.0                       0.0    1.0
5                           1.0     34.0           6.0  ...             

In [8]:
print(data_adult.feature_names, sep=",")

['age', 'education-num', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'workclass=Federal-gov', 'workclass=Local-gov', 'workclass=Private', 'workclass=Self-emp-inc', 'workclass=Self-emp-not-inc', 'workclass=State-gov', 'workclass=Without-pay', 'education=10th', 'education=11th', 'education=12th', 'education=1st-4th', 'education=5th-6th', 'education=7th-8th', 'education=9th', 'education=Assoc-acdm', 'education=Assoc-voc', 'education=Bachelors', 'education=Doctorate', 'education=HS-grad', 'education=Masters', 'education=Preschool', 'education=Prof-school', 'education=Some-college', 'marital-status=Divorced', 'marital-status=Married-AF-spouse', 'marital-status=Married-civ-spouse', 'marital-status=Married-spouse-absent', 'marital-status=Never-married', 'marital-status=Separated', 'marital-status=Widowed', 'occupation=Adm-clerical', 'occupation=Armed-Forces', 'occupation=Craft-repair', 'occupation=Exec-managerial', 'occupation=Farming-fishing', 'occupation=Handlers-cleane

The above data is not the preprocessed or cleaned data. There are 100 features! AIF360 presents us with a function that loads the cleaned version of the Adult dataset and it is much better to handle it.

In [9]:
priv_group = [{'sex':1}]
unpriv_group = [{'sex':0}]

We will address the problem of fairness for gender here. So, we choose the protected group as sex with the males indicated by 1 as the privileged group and the females indicated by 0 as the unprivileged group as shown above.

In [14]:
data_adult = load_preproc_data_adult(['sex'])
data_adult

               instance weights features                      ...                                        labels
                                         protected attribute  ...                                              
                                    race                 sex  ... Education Years=<6 Education Years=>12       
instance names                                                ...                                              
0                           1.0      0.0                 1.0  ...                0.0                 0.0    0.0
1                           1.0      1.0                 1.0  ...                0.0                 0.0    0.0
2                           1.0      1.0                 1.0  ...                0.0                 0.0    1.0
3                           1.0      0.0                 1.0  ...                0.0                 0.0    1.0
4                           1.0      1.0                 0.0  ...                0.0                 0.0

In [15]:
print(data_adult.feature_names, sep=", ")

['race', 'sex', 'Age (decade)=10', 'Age (decade)=20', 'Age (decade)=30', 'Age (decade)=40', 'Age (decade)=50', 'Age (decade)=60', 'Age (decade)=>=70', 'Education Years=6', 'Education Years=7', 'Education Years=8', 'Education Years=9', 'Education Years=10', 'Education Years=11', 'Education Years=12', 'Education Years=<6', 'Education Years=>12']


The number of features in this cleaned pre-processed dataset is far less than the number of features in the original dataset. It is more managable.

### EVALUATION OF FAIRNESS OF DATA 

Here, we will try to find out how fair is the data with respect to the protected group i.e. sex as per the various definitions of fairness.

#### CLEANED DATA FAIRNESS

In [16]:
ad_metrics = BinaryLabelDatasetMetric(data_adult,
                                      unprivileged_groups=unpriv_group,
                                     privileged_groups=priv_group)

- Statistical Parity Difference

In [17]:
ad_metrics.statistical_parity_difference()

-0.19451574596420296

We observe quite a bit of deviation from 0 which is in favor of the male group. Hence, data is not fair as per this definition. 

- Disparate Impact

In [18]:
ad_metrics.disparate_impact()

0.3596552625800337

We see that this value is clearly not close to 1 rather it is close to 0 again showing that the male individuals are favored in the data compared to females.

- Consistency

In [25]:
ad_metrics.consistency()[0]

0.7566602514229388

The value is not very close to 1, so somewhat the individual fairness is also not exhibited properly by the data.

#### FAIRNESS OF TRAINED MODEL